In [1]:
import pandas as pd
from weibo import weibo90s
from weibo import weibofans30
from pyecharts import Bar, Line, Graph, Overlap, WordCloud, EffectScatter

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "/Users/linhuiqing/venv/lib/python3.7/site-packages/lml/utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [2]:
df_tw = pd.read_csv('dataset/results_tw.csv', encoding = 'gbk').fillna('')
df_hk = pd.read_csv('dataset/results_hk.csv', encoding = 'gbk').fillna('')
df_ml = pd.read_csv('dataset/results_ml.csv', encoding = 'gbk').fillna('')

In [3]:
df_concat = pd.concat([df_tw, df_hk, df_ml], ignore_index = True).drop_duplicates()

# 1

In [4]:
def year_quantity(df):
    years = []
    numbers = []
    for i in df['年份'].drop_duplicates().sort_values():
        tmp = df[df['年份'] == i]
        years.append(i)
        numbers.append(len(tmp))
    results = pd.DataFrame({'年份': years, '总部数': numbers})
    return results

In [5]:
def year_quality(df):
    years = []
    scores = []
    for i in df['年份'].drop_duplicates().sort_values():
        tmp = df[df['年份'] == i]
        score = tmp['豆瓣评分'].sum() / len(tmp)
        years.append(i)
        scores.append(round(score, 2))
    results = pd.DataFrame({'年份': years, '平均分': scores})
    return results

In [6]:
def display_line(df, area):
    x = df['年份'].drop_duplicates().sort_values().apply(str)
    y1 = year_quantity(df)['总部数']
    y2 = year_quality(df)['平均分']
    line = Line(area)
    line.add('总部数', x, y1, mark_line = ['average'])
    line.add('平均分', x, y2, mark_line = ['average'])
    return line

In [7]:
display_line(df_tw, '台湾')

In [8]:
display_line(df_hk, '香港')

In [9]:
display_line(df_ml, '大陆')

In [10]:
display_line(df_concat, '总体')

# 2

In [11]:
tmp = df_concat[df_concat['年份'] > 2012]

In [12]:
tmp[tmp['评价人数/人'] > 100000].sort_values(by = '豆瓣评分', ascending = True).head(3)

,剧名,年份,导演,编剧,主演,类型,制片国家/地区,集数,单集片长/分钟,豆瓣评分,评价人数/人
2154,深夜食堂,2017,蔡岳勋 / 胡涵清,陈怡方,黄磊 / 赵又廷 / 张钧甯 / 海清 / 何炅 / 陈意涵 / 马苏 / 刘昊然 / 萧敬...,剧情,中国大陆,36,45,2.8,104240
2155,漂亮的李慧珍,2017,赵晨阳,卢至柔 / 杨青,迪丽热巴 / 盛一伦 / 李溪芮 / 张彬彬 / 王骁 / 王一楠 / 任伟 / 王一菲 /...,喜剧 / 爱情,中国大陆,40,43,2.8,109354
2321,青云志,2016,朱锐斌 / 周远舟 / 刘国辉 / 朱少杰,邵潇逸 / 张少微 / 张戬,李易峰 / 赵丽颖 / 杨紫 / 舒畅 / 王源 / 王俊凯 / 易烊千玺 / 熊乃瑾 / ...,奇幻 / 武侠 / 古装,中国大陆,55,45,5.3,122359


In [13]:
tmp[tmp['评价人数/人'] > 100000].sort_values(by = '豆瓣评分', ascending = False).head(3)

,剧名,年份,导演,编剧,主演,类型,制片国家/地区,集数,单集片长/分钟,豆瓣评分,评价人数/人
2436,琅琊榜,2015,孔笙 / 李雪,海宴,胡歌 / 刘涛 / 王凯 / 陈龙 / 黄维德 / 靳东 / 高鑫 / 张_琰 / 张龄心 ...,剧情 / 古装,中国大陆,54,45,9.2,256604
1941,白夜追凶,2017,王伟,指纹 / 顾小白,潘粤明 / 王泷正 / 梁缘 / 吕晓霖 / 尹姝贻 / 侯雪龙 / 宋乃刚 / 王余阳 /...,剧情 / 悬疑 / 犯罪,中国大陆,32,45,9.0,245310
2444,伪装者,2015,李雪,张勇,胡歌 / 靳东 / 刘敏涛 / 王凯 / 宋轶 / 王乐君 / 王鸥 / 刘奕君 / 岳_ ...,剧情 / 悬疑,中国大陆,48,40,8.4,100309


# 3

In [14]:
def style_quantity(df):
    styles = []
    for i in df['类型']:
        style = i.split(' / ')
        for j in style:
            if j != '':
                styles.append(j)
    tmp = pd.DataFrame({'styles': styles})['styles'].value_counts()
    results = pd.DataFrame({'类型': tmp.index, '总部数': tmp.values})
    results.drop(0, inplace = True)
    return results

In [15]:
def display_bar(df, area):
    x = style_quantity(df)['类型'][:10]
    y = style_quantity(df)['总部数'][:10]
    bar = Bar(area)
    bar.add('总部数', x, y, is_label_show = True, xaxis_interval = 0)
    return bar

In [16]:
display_bar(df_tw, '台湾')

In [17]:
display_bar(df_hk, '香港')

In [18]:
display_bar(df_ml, '大陆')

In [51]:
name = style_quantity(df_concat)['类型']
value = style_quantity(df_concat)['总部数']
wordcloud = WordCloud('总体')
wordcloud.add('', name, value, shape = 'circle', word_size_range = [20, 100])

# 4

In [24]:
def actor_score_number(df):
    actors = []
    scores = []
    for i in df.index:
        tmp = (df['主演'][i].split(' / '), df['豆瓣评分'][i])
        for actor in tmp[0]:
            if actor != '':
                actors.append(actor)
                scores.append(tmp[1])
    datas = pd.DataFrame({'actors': actors, 'scores': scores})
    actors = datas.actors.drop_duplicates()
    my1 = []
    my2 = []
    my3 = []
    for actor in actors:
        my1.append(actor)
        tmp = datas[datas.actors == actor]
        my2.append(round(tmp.scores.sum() / len(tmp), 2))
        my3.append(len(tmp))
    results = pd.DataFrame({'演员': my1, '平均分': my2, '部数': my3}).sort_values('平均分', ascending = False)
    return results

In [21]:
# actor_score_number(df_concat).to_csv('dataset/演员-平均分-部数.csv', index = False, encoding = 'gbk')

In [25]:
df = pd.read_csv(open('dataset/演员-平均分-部数.csv',encoding = 'gbk'))

In [102]:
wb90s = pd.DataFrame({'演员': [item[0] for item in weibo90s().items()], '粉丝数': [item[1] for item in weibo90s().items()]})
wbfans30 = pd.DataFrame({'演员': [item[0] for item in weibofans30().items()], '粉丝数': [item[1] for item in weibofans30().items()]})
wb90s = pd.merge(wb90s, df)
wbfans30 = pd.merge(wbfans30, df)

In [103]:
# wbfans30 = wbfans30.sort_values(by = '部数',ascending = False)
# wb90s = wb90s.sort_values(by = '部数',ascending = False)

In [110]:
def display_overlap(df, name):
    x = df['演员']
    y1 = df['部数']
    y2 = df['平均分']
    bar = Bar(name, '注：横坐标依据粉丝数从大到小排列')
    line = Line()
    bar.add('部数', x, y1, xaxis_interval = 0, is_datazoom_show = True, datazoom_range = [0, 50])
    line.add('平均分', x, y2, is_label_show = True)
    overlap = Overlap()
    overlap.add(bar)
#     overlap.add(line)
    overlap.add(line, yaxis_index=1, is_add_yaxis=True)
    return overlap



In [112]:
display_overlap(wb90s, '微博九零后')

In [113]:
display_overlap(wbfans30, '微博粉丝榜')

# 5

In [30]:
df_hot = df_concat[df_concat['评价人数/人'] > 100000]

In [31]:
def actor_number(df):
    actors = []
    for i in df['主演']:
        tmp = i.split(' / ')
        for actor in tmp:
            if (actor != '') & (actor.find('_') == -1):
                actors.append(actor)
    datas = pd.DataFrame({'actors': actors})
    results = pd.DataFrame({'演员': datas.actors.value_counts().index, '部数': datas.actors.value_counts().values})
    results = results[results['部数'] > 1]
    return results

In [32]:
def find_nodes(df):
    tmp = actor_number(df)
    nodes = []
    for i in tmp.index:
        dic = {}
        dic['name'] = tmp['演员'][i]
        dic['value'] = tmp['部数'][i]
        dic['symbolSize'] = tmp['部数'][i] * 3
        nodes.append(dic)
    return nodes

In [33]:
nodes_hot = find_nodes(df_hot)

In [34]:
def actor_list(df):
    results = []
    for actors in df['主演']:
        if actors != '':
            result = []
            for actor in actors.split(' / '):
                if (actor != '') & (actor.find('_') == -1):
                    result.append(actor)
            results.append(result)
    return results

In [35]:
def find_df(df):
    tmp = actor_list(df)
    source = []
    target = []
    for actors in tmp:
        for i in range(len(actors)-1):
            for j in actors[i+1:]:
                source.append(actors[i])
                target.append(j)
    results = pd.DataFrame({'source': source, 'target': target})
    return results

In [36]:
links_hot = []
for index, row in find_df(df_hot).iterrows():
    source = row.source
    target = row.target
    links_hot.append({'source': source, 'target': target})

In [39]:
def display_graph(nodes, links):
    graph = Graph('热门电视剧演员关系图')
    graph.add('', nodes, links)
    return graph

In [41]:
display_graph(nodes_hot, links_hot).render('热门电视剧演员关系图.html')

# 6

In [42]:
datas = pd.read_csv('dataset/tmp.csv', encoding = 'gbk')
results = pd.DataFrame(index = datas['年份'].drop_duplicates().sort_values())

In [43]:
def add_column(area):
    column = []
    for year in results.index:
        column.append(len(datas[(datas['年份'] == year) & (datas['制片国家/地区'] == area)]))
    results[area] = column

In [44]:
areas = ['香港 / 台湾', '中国大陆 / 香港', '中国大陆 / 台湾', '中国大陆 / 香港 / 台湾']
for area in areas:
    add_column(area)

In [45]:
results

,香港 / 台湾,中国大陆 / 香港,中国大陆 / 台湾,中国大陆 / 香港 / 台湾
年份,,,,
1986,0,2,0,0
1990,0,1,0,0
1991,0,0,1,0
1993,1,1,0,1
1994,0,0,0,1
1995,1,1,2,0
1996,1,4,1,2
1997,0,3,0,0
1998,1,2,5,1


In [46]:
es = EffectScatter('三地合作情况图', '注：纵坐标代表合作程度，值越大表示合作次数越多')
x = results.index
def add_es(area):
    values = []
    for value in results[area].values:
        if value == 0:
            values.append(None)
        else:
            values.append(value)
    y = values
    es.add(area, x, y, xaxis_min = 1985, xaxis_max = 2020, xaxis_force_interval = 5)

In [47]:
for area in areas:
    add_es(area)

In [48]:
es

# 7